# Crop Rotation and Production Planning

## Modelling with Julia/JuMP

## TODO

* Add general intro to Julia/JuMP, show math/latex/juMP side by side (as slides).
* Should we do this as slides/walkthrough/play along at home?
* Crop rotation problem has more complexity that could be added.
* Area assignment problem always produces single crop schedules (fixing above will probably help).

# Additional Constraints

* Rotation example over 12 months.
* Restricted planting times (range - index variables).
* Single column generation model - minimise used area with naive initial columns.

# Part 1: Generating Feasible Crop Schedules

Our model defines feasible crop rotation schedules on a single plot.

The constraints enforce that:
* only one crop is planted in each period (with variable planting durations),
* crops of the same family do not follow one another,
* one green manuring crop is planted, and
* there is one period of fallow.

(This is a simplified model with no restrictions on planting times).

## Mixed Integer Programming Model

This set of variables and constraints defines all feasible crop rotation schedules.

\begin{alignat*}{2}
& \sum_{i=1}^{n} \sum_{r=0}^{t_i-1} x_{i,j-r} \le 1, \quad j = 1 \dots M, \\
& \sum_{i \in F_p} \sum_{r=0}^{t_i} x_{i,j-r} \le 1, \quad p = 1 \dots NF,\, j = 1 \dots M, \\
& \sum_{i\in G} \sum_{j=1}^{M} x_{i,j} \le 1, \\
& \sum_{j=1}^{M} x_{nj} = 1, \\
& x_{ij} \in \lbrace 0, 1 \rbrace, \quad i = 1 \dots n,\, j = 1 \dots M  \\
\end{alignat*}

## Solving the Model with JuMP

JuMP provides the general MIP modelling interface.

In [2]:
using JuMP

In this case, Clp/Cbc handle the solving the problems we formulate.
This can be replaced by any suitable solver:

* GPLK
* Gurobi
* SCIP
* ...

In [3]:
using Clp, Cbc

┌ Info: Precompiling Clp [e2554f3b-3117-50c0-817c-e040a3ddf72d]
└ @ Base loading.jl:1192


## Problem Data

First define our problem data using Julia arrays.

Those who've used MATLAB or Python before should find this syntax quite familiar.

In [6]:
M = 8                 # Periods in the rotation.
C = 1:2               # Productive crops.
G = [3]               # Green manuring crops.
N = 4                 # Total crop types including fallow period.
F = [[1, 2], [3]]     # Crop families.
t = [2, 3, 2, 1];     # Time to yield for each crop.

### Modelling with Binary Variables

Begin the model with binary variables $x_{ij}$ (crop i planted in period j).

In [17]:
model = Model(solver=CbcSolver())
@variable(model, x[i in 1:N, j in 1:M], Bin)

4×8 Array{Variable,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  x[1,6]  x[1,7]  x[1,8]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]  x[2,6]  x[2,7]  x[2,8]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]  x[3,6]  x[3,7]  x[3,8]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]  x[4,6]  x[4,7]  x[4,8]

### No overlap of crop plantings

In [29]:
j = 1
@constraint(model, sum(
    sum(x[i, mod(j - r - 1, M) + 1] for r in 0:t[i]-1)
    for i in 1:N) <= 1)

x[1,1] + x[1,8] + x[2,1] + x[2,8] + x[2,7] + x[3,1] + x[3,8] + x[4,1] ≤ 1

Crops 1 and 3 last for two periods, while crop 2 lasts for three periods.
This constraint correctly specifies that the following choices are mutually exclusive:
* Planting crop 1 in periods 1 or 8
* Planting crop 2 in periods 1, 7, or 8
* Planting crop 3 in periods 1 or 8
* Planting no crop (fallow) to period 1

The following code generates these constraints for every planting period:

In [19]:
@constraint(
    model, [j in 1:M],
    sum(
        sum(x[i, mod(j - r - 1, M) + 1] for r in 0:t[i]-1)
        for i in 1:N) <= 1);

### Crops of the same family may not be planted after one another.

Inspect the expression generated for period 1 and family 1 (containing production crops 1 and 2).

In [20]:
p = 1
j = 1
@constraint(
    model, sum(
        sum(x[i, mod(j - r - 1, M) + 1] for r in 0:t[i])
        for i in F[p]) <= 1)

x[1,1] + x[1,8] + x[1,7] + x[2,1] + x[2,8] + x[2,7] + x[2,6] ≤ 1

This expression extends the crop overlap constraint *for this specific family of crops* by one period, thus enforcing a break of at least one period between plantings of these crops.
Therefore the following additional choices are mutually exclusive:
* Planting crop 1 in periods 1, 7, or 8
* Planting crop 2 in periods 1, 6, 7, or 8

This constraint is added to the model for every planting period and every crop family:

In [21]:
@constraint(
    model, [p in 1:length(F), j in 1:M],
    sum(
        sum(x[i, mod(j - r - 1, M) + 1] for r in 0:t[i])
        for i in F[p]) <= 1);

### One green manure crop and one period of fallow required.

In [22]:
@constraint(model, sum(sum(x[i, j] for j in 1:M) for i in G) == 1)

x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] + x[3,6] + x[3,7] + x[3,8] = 1

In [23]:
@constraint(model, sum(x[N, j] for j in 1:M) == 1)

x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] + x[4,6] + x[4,7] + x[4,8] = 1

## Solve with a simple objective function.

Below assigns a simple objective function to maximise weighted production.
Since we used a small example data set we can inspect the generated model to see if our expressions are correct.

In [24]:
@objective(model, Max, sum(x[1,j] for j in 1:M) + 2 * sum(x[2,j] for j in 1:M));

## Solve the resulting model and display the solution

In [10]:
solve(model)

:Optimal

In [11]:
# Return a string representation of the resulting crop planting times.
function get_schedule(x, M, N)
    schedule = ""
    for j in 1:M
        selected_crop = [i for i in 1:N if getvalue(x[i, j]) > 0.999]
        if length(selected_crop) > 0
            if selected_crop[1] == N
                schedule = string(schedule, "|", "F")
            else
                schedule = string(schedule, "|", selected_crop[1])
            end
        else
            schedule = string(schedule, "| ")
        end
    end
    schedule = string(schedule, "|")
end

# Returns the number of plantings of each crop in this schedule.
function get_production(x, M, C)
    [sum(getvalue(x[i, j]) for j in 1:M) for i in C]
end

println("Schedule:   ", get_schedule(x, M, N))
println("Production: ", get_production(x, M, C))
            

Schedule:   | |F|1| |3| |2| |
Production: [1.0, 1.0]


# Part 2: Assigning schedules to plot areas

Generate crop schedules $k \in K$ which produce $c_{ik}$ of crop $i$ per unit area.
In the phase 1 problem we assign areas $a_k$ to schedules in order to minimise uncovered demand.

The restricted master problem for the current set of columns is:

\begin{alignat*}{2}
& \min \,\, \sum_{i \in C} s_{i} \\
& \sum c_{ik} a_{k} + s_{i} \ge d_{i}, \quad i \in C \\
& a_{k} \ge 0, \quad k \in K \\
& s_{i} \ge 0, \quad i \in C \\
\end{alignat*}

The subproblem searches for feasible crop schedules with the following objective:

\begin{alignat*}{2}
& \max \,\, \sum_{i \in C} \sum_{i=1}^{m} \lambda_{i} x_{ij} \\
\end{alignat*}

where $\lambda_i$ are the dual variables associated with the demand coverage constraints.

In [12]:
# Crop rotation schedule data.
M = 12     # Periods in the rotation.
C = 1:6    # Production crops.
G = 7:8    # Green manuring crops.
N = 9      # Total crop types including fallow period.
F = [[1, 2], [3, 4], [5, 6], [7, 8]]   # Crop families 1-4.
t = [4, 3, 3, 4, 3, 3, 3, 2, 1]        # Time to yield.

# Demand to be satisfied.
d = [1, 1, 1, 1, 0, 0];

## Define pricing problem as a callable function

In [13]:
# Construct and solve a crop schedule feasibility model with
# the given objective values applied to each crop type.
function price_crop_rotation(L)
    model = Model(solver=CbcSolver())
    @variable(model, x[i in 1:N, j in 1:M], Bin)
    @constraint(
        model, single_crop[j in 1:M],
        sum(
            sum(x[i, mod(j - r - 1, M) + 1] for r in 0:t[i]-1)
            for i in 1:N) <= 1)
    @constraint(
        model, crop_family[p in length(F), j in 1:M],
        sum(
            sum(x[i, mod(j - r - 1, M) + 1] for r in 0:t[i])
            for i in F[p]) <= 1)
    @constraint(
        model, green_manure,
        sum(sum(x[i, j] for j in 1:M) for i in G) == 1)
    @constraint(
        model, fallow,
        sum(x[N, j] for j in 1:M) == 1)
    @objective(model, Max, sum(L[i] * x[i, j] for i in C for j in 1:M))
    solve(model)
    getobjectivevalue(model), get_production(x, M, C), get_schedule(x, M, N)
end

price_crop_rotation (generic function with 1 method)

### Generate a column to start with.

We are solving for a feasible area assignment first, so we generate any old column.
In this case, we get a schedule which plants crops 2 and 3.
We'll start solving the restricted master problem using this single column.

In [14]:
# Create an initial column and store the associated data.
# The pricing problem gives feasible crop rotation schedules
# so we can use any objective function to generate columns.
objective, production, schedule = price_crop_rotation(d)
columns = [Dict("production" => production, "schedule" => schedule)]

1-element Array{Dict{String,Any},1}:
 Dict("production"=>[0.0, 2.0, 1.0, 0.0, 0.0, 0.0],"schedule"=>"|2| | |3| | |F|2| | |8| |")

### Construct the initial master problem with the available crop schedules.

In [15]:
master_model = Model(solver=ClpSolver())
@variable(master_model, a[i in 1:length(columns)] >= 0)
@variable(master_model, s[j in 1:length(d)] >= 0)

# Capture the area variables along with the stored column data.
for i in 1:length(columns)
    columns[i]["area"] = a[i]
end

# Uses the production associated with each column to generate coverage constraint.
# Note that this creates an object demand_covered which represents the constraint;
# we can use this to get dual values later.
@constraint(
    master_model, demand_covered[j in 1:length(d)],
    sum(columns[i]["production"][j] * a[i] for i in 1:length(columns)) + s[j] >= d[j])
@objective(master_model, Min, sum(s))
master_model

Minimization problem with:
 * 6 linear constraints
 * 7 variables
Solver is Clp

### Obtain the dual values and solve the pricing problem to generate a new column.

An extra schedule is added which produces crop 1, resulting in a new variable z added to the master problem which can satisfy demand for that crop.

In [16]:
solve(master_model)
objective, production, schedule = price_crop_rotation(getdual(demand_covered))
println(production)
println(schedule)
@variable(
    master_model, z >= 0, objective=0.0,
    inconstraints=demand_covered, coefficients=production)
# Store the master variable and associated data of the new column.
push!(columns, Dict("production" => production, "schedule" => schedule, "area" => z))
master_model

[2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
|F|1| | | | |8| |1| | | |


Minimization problem with:
 * 6 linear constraints
 * 8 variables
Solver is Clp

### Iterate until no improving column can be generated

The loop below runs the column generation loop to optimality, i.e. until the optimal objective value of the pricing problem is less than or equal to 1.

In [17]:
while true
    solve(master_model)
    objective, production, schedule = price_crop_rotation(getdual(demand_covered))
    if objective <= 1
        display("DONE")
        break
    end
    @variable(
        master_model, z >= 0, objective=0.0,
        inconstraints=demand_covered, coefficients=production)
    new_column = Dict("production" => production, "schedule" => schedule, "area" => z)
    push!(columns, new_column)
    display("COLUMN ADDED")
    display(new_column)
end

"COLUMN ADDED"

Dict{String,Any} with 3 entries:
  "production" => [0.0, 0.0, 0.0, 2.0, 0.0, 0.0]
  "area"       => z
  "schedule"   => "| |4| | | |F|4| | | |8| |"

"DONE"

### View the resulting schedules

We can see that this assignment over-covers demand for crop 2, so we may be able to add additional columns to reduce area.

In [18]:
for column in columns
    println(column["schedule"], "   AREA = ", getvalue(column["area"]))
end

|2| | |3| | |F|2| | |8| |   AREA = 1.0
|F|1| | | | |8| |1| | | |   AREA = 0.5
| |4| | | |F|4| | | |8| |   AREA = 0.5


## From this feasible solution, minimise area

Modify the restricted master problem to:

\begin{alignat*}{2}
& \min \,\, \sum_{k \in K} a_{k} \\
& \sum c_{ik} a_{k} \ge d_{i}, \quad i \in C \\
& a_{k} \ge 0, \quad k \in K \\
\end{alignat*}

`@objective` can be used to alter the objective in the current master problem, while an additional constraint is added which fixes the uncovered demand variables to zero.
This allows the current state of the problem to be kept (simplex will restart from our current feasible basis).

In [19]:
# Alter the objective to minimise area.
@objective(master_model, Min, sum(column["area"] for column in columns))
@constraint(master_model, [j in 1:length(d)], s[j] == 0)

# Run column generation loop to optimality.
while true
    solve(master_model)
    objective, production, schedule = price_crop_rotation(getdual(demand_covered))
    if objective <= 1
        display("DONE")
        break
    end
    @variable(
        master_model, z >= 0, objective=0.0,
        inconstraints=demand_covered, coefficients=production)
    new_column = Dict("production" => production, "schedule" => schedule, "area" => z)
    push!(columns, new_column)
    display("COLUMN ADDED")
    display(new_column)
end

"COLUMN ADDED"

Dict{String,Any} with 3 entries:
  "production" => [0.0, 0.0, 3.0, 0.0, 0.0, 0.0]
  "area"       => z
  "schedule"   => "|3| | |F|8| |3| | |3| | |"

"COLUMN ADDED"

Dict{String,Any} with 3 entries:
  "production" => [0.0, 3.0, 0.0, 0.0, 0.0, 0.0]
  "area"       => z
  "schedule"   => "|2| | |F|8| |2| | |2| | |"

"DONE"

### Final result

The second column generation loop adds new production schedules for crops 2 and 3.
We get a boring solution though...

In [20]:
for column in columns
    println(column["schedule"], "   AREA = ", getvalue(column["area"]))
end

|2| | |3| | |F|2| | |8| |   AREA = 0.0
|F|1| | | | |8| |1| | | |   AREA = 0.5
| |4| | | |F|4| | | |8| |   AREA = 0.5
|3| | |F|8| |3| | |3| | |   AREA = 0.3333333333333333
|2| | |F|8| |2| | |2| | |   AREA = 0.3333333333333333
